In [ ]:
from types import SimpleNamespace
import numpy as np
import deepdish as dd
import os
import holoviews as hv
import xarray as xr
hv.notebook_extension()
%opts Scatter (color='b')

# No disorder

In [ ]:
path = './'
files = [f for f in os.listdir(path) if f.startswith('no_disorder_B_vs_I_c')]

fs = {f: dd.io.load(path + f) for f in files}

hm_I = {}
hm_phases = {}

for fname, data in fs.items():
    wire_params = data['wire_params']
    p = data['p']
    key = (wire_params['a'], wire_params['L'], data['T'],
           p.mu, wire_params['with_leads'], p.orbital, p.g, p.alpha)
    I_cs = data['I_cs']
    Bs = data['B_xs']
    phases = data['phases']
    
    hm_I[key] = hv.Curve((Bs, I_cs), kdims=[hv.Dimension('$B$', unit='T')], vdims=['$I$'])
    hm_phases[key] = hv.Curve((Bs, phases), kdims=[hv.Dimension('$B$', unit='T')],
                              vdims=[hv.Dimension('$\phi$', unit='rad')])
    

kdims = [hv.Dimension('$a$', unit='nm'),
                      hv.Dimension('$L$', unit='nm'),
                      hv.Dimension('$T$', unit='K'),
                      hv.Dimension('$\mu$', unit='meV'),
                      'infinite leads', 'orbital', 'g-factor', 'alpha']

no_disorder_I = hv.HoloMap(hm_I, kdims=kdims)
no_disorder_phases = hv.HoloMap(hm_phases, kdims=kdims)

In [ ]:
table = no_disorder_I.table()

In [ ]:
table.dframe()

In [ ]:
(no_disorder_I + no_disorder_phases.select(B=(0, 1))).cols(1)

In [ ]:
len(files)

# Disorder

In [ ]:
def disorder_B_vs_I(fname):
    file = dd.io.load(fname)

    print(file.keys())

    p = file['p']
    T = file['T']
    tol = file['tol']
    constants = file['constants']
    Bs = file['B_xs']
    phases = file['phases']
    wire_params = file['wire_params']
    I_c = file['I_cs']
    disorders = file['disorders']
    salts = file['salts']

    curves = {}
    for i, realisation in enumerate(I_c):
        for disorder, I_cs in zip(disorders, realisation):
            curves[(i, disorder)] = hv.Curve((Bs, I_cs), kdims=[hv.Dimension('$B$', unit='$T$')], vdims=['$I_c$'])
    hm = hv.HoloMap(curves, kdims=['salt', 'disorder'])

    return hm


def collapse_salts(hm):
    collapsed = hm.collapse('salt', function=np.mean, spreadfn=np.std)
    disorder = collapsed * collapsed.table().to.spread(['$B$', '$I_c$'], '$I_c$_std')
    return disorder

In [ ]:
path = './'
files = [f for f in os.listdir(path) if f.startswith('disorder_B_vs_I_c')]

fs = {f: dd.io.load(path + f) for f in files}

hm = {}

    
for fname, data in fs.items():
    disorders = data['disorders']
    I_cs = data['I_cs']
    Bs = data['B_xs']
    wire_params = data['wire_params']
    p = data['p']
    dims = dict(kdims=[hv.Dimension('$B$', unit='$T$')], vdims=['$I_c$'])

    for disorder, I_cs, std in zip(disorders, I_cs.mean(axis=0), I_cs.std(axis=0)):
        key = (wire_params['a'], wire_params['L'], data['T'], 
               p.mu, disorder, wire_params['with_leads'], p.orbital, p.g)
        val = hv.Spread((Bs, I_cs, std), kdims=[hv.Dimension('$B$', unit='$T$')], vdims=[hv.Dimension('$I_c$'), 'err'])
        val *= hv.Path((Bs, I_cs), kdims=[hv.Dimension('$B$', unit='$T$'), '$I_c$'])
        hm[key] = val[:, 0:100]


hv.HoloMap(hm, kdims=[hv.Dimension('$a$', unit='nm'),
                      hv.Dimension('$L$', unit='nm'),
                      hv.Dimension('$T$', unit='K'),
                      hv.Dimension('$\mu$', unit='meV'),
                      hv.Dimension('disorder', unit='meV'),
                      'infinite leads', 'orbital', 'zeeman']).select(T=1)

In [ ]:
len(files)

# Mean free path $g \sim N_{ch} / (1 + L / \lambda_{MFP})$
Check with Carlo's review paper on random matrix theory

In [ ]:
def mfp_hms(fname):
    mfp = dd.io.load(fname)
    disorders = mfp['disorders']
    conductances = xr.DataArray(mfp['conductances'], dims=['length', 'salts', 'disorder'])
    Ls = np.array(mfp['Ls'])
    num_propagating = np.unique(mfp['num_propagating'])
    assert len(num_propagating) == 1

    hm_dict = {}
    for L, conductance in zip(Ls, conductances):
        conductance_mean = conductance.mean(dim='salts')
        curve = hv.Curve((disorders, conductance_mean), kdims=['disorder'], vdims=['conductance'])
        spread = hv.Spread((disorders, conductance_mean, conductance.std(dim='salts')))
        scatter = hv.Scatter((disorders, conductance_mean))
        hm_dict[L] = curve * spread * scatter
    hm_disorder_vs_g = hv.HoloMap(hm_dict, kdims=['L'])

    hm_dict = {}
    for disorder, resistance in zip(disorders, (1 / conductances.T)):
        resistance_mean = resistance.mean(dim='salts')
        curve = hv.Curve((Ls, resistance_mean), kdims=['L'], vdims=['resistance'])
        spread = hv.Spread((Ls, resistance_mean, resistance.std(dim='salts')))
        scatter = hv.Scatter((Ls, resistance_mean))
        hm_dict[disorder] = curve * spread * scatter
    hm_L_vs_R = hv.HoloMap(hm_dict, kdims=['disorder'])

    fits = np.squeeze([np.polyfit(Ls, resistance, deg=1) for resistance in (1/conductances).mean('salts').transpose('disorder', 'length')])
    polyfits = [np.poly1d(fit)(Ls) for fit in fits]
    l_mfps = 1 / (num_propagating[0] * fits[:, 0])
    
    dims = dict(kdims=['L'], vdims=['resistance'])
    hm_fit = hv.HoloMap({disorder: hv.Curve((Ls, R), **dims) for disorder, R in zip(disorders, polyfits)}, 
           kdims=['disorder'])
          
    print("")
    print("with {} modes at {} meV".format(num_propagating[0], mfp['p'].mu))
    for disorder, l_mfp in zip(disorders, l_mfps):
        print("Disorder strength: {} meV and mean free path: {:.0f} nm".format(disorder, l_mfp))
    
    return hm_disorder_vs_g, hm_L_vs_R, hm_fit

hm_disorder_vs_g_40meV, hm_L_vs_R_40meV, hm_fit_40meV = mfp_hms('data/mfp_mu-40meV.h5')
hm_disorder_vs_g_20meV, hm_L_vs_R_20meV, hm_fit_20meV = mfp_hms('data/mfp_mu-20meV.h5')
hm_disorder_vs_g_15meV, hm_L_vs_R_15meV, hm_fit_15meV = mfp_hms('data/mfp_mu-15meV.h5')
hm_disorder_vs_g_6meV, hm_L_vs_R_6meV, hm_fit_6meV = mfp_hms('data/mfp_mu-6meV.h5')

In [ ]:
hm_disorder_vs_g_15meV.relabel("mu = 15 meV, 6 modes") + hm_disorder_vs_g_6meV.relabel("mu = 6 meV, 2 modes")

In [ ]:
((hm_L_vs_R_15meV * hm_fit_15meV).relabel("mu = 15 meV, 6 modes")  +
 (hm_L_vs_R_6meV * hm_fit_6meV).relabel("mu = 6 meV, 2 modes"))

# $T$ vs $I_c$ with disorder

In [ ]:
def T_vs_Ic(fname):
    dat = dd.io.load(fname)

    Ts = dat['T']
    I_cs = xr.DataArray(dat['I_cs'], dims=['T', 'disorder', 'salts'])
    disorders = dat['disorders']

    I_cs_mean = I_cs.mean('salts')

    dims = dict(kdims=['$T$'], vdims=['$I_c$'])
    hm = hv.HoloMap({disorder: hv.Scatter((Ts, I), **dims) for disorder, I in zip(disorders, I_cs_mean.T)}, 
               kdims=['disorder'])
    return hm

(T_vs_Ic('data/T_vs_Ic_L_600.h5').relabel("mu = 15 meV, 12 modes") +
 T_vs_Ic('data/T_vs_Ic_L_600_mu-6meV.h5').relabel("mu = 6 meV, 4 modes") )